# ROS and Unit Test

The ROS unittest example was from the Duckietown materials.

## Installation and Setup

### Install ROS 

We will use ROS melodic with Python 2

In [ ]:
%%bash
sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
apt update  &> /dev/null
apt install ros-melodic-ros-base &> /dev/null

sudo rosdep init  &> /dev/null
rosdep update  &> /dev/null

apt install python-rosinstall  &> /dev/null
apt install python-catkin-tools  &> /dev/null
apt install ros-melodic-ros-numpy ros-melodic-vision-opencv &> /dev/null

Executing: /tmp/apt-key-gpghome.p3URytLwJ8/gpg.1.sh --keyserver hkp://keyserver.ubuntu.com:80 --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654


gpg: key F42ED6FBAB17C654: public key "Open Robotics <info@osrfoundation.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!sudo apt-get install libeigen3-dev ros-melodic-theora-image-transport ros-melodic-image-publisher

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libeigen3-dev is already the newest version (3.3.4-4).
ros-melodic-image-publisher is already the newest version (1.15.0-1bionic.20210921.211325).
ros-melodic-theora-image-transport is already the newest version (1.9.5-0bionic.20210921.211317).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.


### Setup Python Path

So that we could use rospy, rosbag etc in the notebook.


In [ ]:
import sys
import os

sys.path.append('/opt/ros/melodic/lib/python2.7/dist-packages/')
print(sys.path)

os.environ['PATH'] += ':/opt/ros/melodic/bin'
print(os.environ['PATH'])

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/opt/ros/melodic/lib/python2.7/dist-packages/', '/opt/ros/melodic/lib/python2.7/dist-packages/']
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/ros/melodic/bin:/opt/ros/melodic/bin


### Create a workspace

In [ ]:
!mkdir -p /content/catkin_ws/src

## Get Ready the AprilTags Repos



### apriltags_ros 

We will use the ROS package, widely used in the robotics community. There are two ROS packages:
* apriltags: the C++ code as library
* apriltags_ros: also C++ with ROS publisher and subscriber.

General Info about April Tags https://april.eecs.umich.edu/wiki/AprilTags

In [ ]:
%cd /content/catkin_ws/src
!git clone https://github.com/Sensing-Intelligent-System/apriltags_ros.git

/content/catkin_ws/src
fatal: destination path 'apriltags_ros' already exists and is not an empty directory.


### apriltags_ros_test

We will use another repo, originally located in the Duckietown repo.

In [ ]:
%cd /content/catkin_ws/src
!git clone https://github.com/Sensing-Intelligent-System/apriltags_ros_test.git

/content/catkin_ws/src
fatal: destination path 'apriltags_ros_test' already exists and is not an empty directory.


### catkin_make

Now we have two ROS packages under src/. Let's compile with catkin_make

In [ ]:
%cd /content/catkin_ws
!source /opt/ros/melodic/setup.bash && catkin_make

/content/catkin_ws
Base path: /content/catkin_ws
Source space: /content/catkin_ws/src
Build space: /content/catkin_ws/build
Devel space: /content/catkin_ws/devel
Install space: /content/catkin_ws/install
####
#### Running command: "make cmake_check_build_system" in "/content/catkin_ws/build"
####
####
#### Running command: "make -j2 -l2" in "/content/catkin_ws/build"
####
[ 45%] Built target apriltags
[ 45%] Built target _apriltags_ros_generate_messages_check_deps_AprilTagDetectionArray
[ 45%] Built target std_msgs_generate_messages_cpp
[ 45%] Built target geometry_msgs_generate_messages_cpp
[ 45%] Built target std_msgs_generate_messages_eus
[ 45%] Built target geometry_msgs_generate_messages_eus
[ 45%] Built target _apriltags_ros_generate_messages_check_deps_AprilTagDetection
[ 45%] Built target std_msgs_generate_messages_py
[ 45%] Built target geometry_msgs_generate_messages_py
[ 45%] Built target std_msgs_generate_messages_nodejs
[ 45%] Built target geometry_msgs_generate_messages_n

## ROS Test

### The tests folder

In the tests filder, again you could see a pair of tester_node (.py and .test). You could also find two images files, which will be loaded during testing.

In [ ]:
!ls /content/catkin_ws/src/apriltags_ros_test/tests

20160508-apriltag108-starducks.png  apriltags_tester_node.py
20210315-apriltag163-stop.jpg	    apriltags_tester_node.test


### apriltags_tester_node.py

In [ ]:
!cat /content/catkin_ws/src/apriltags_ros_test/tests/apriltags_tester_node.py

#!/usr/bin/env python
import rospy
import unittest
import rostest
from apriltags_ros.msg import AprilTagDetectionArray
from sensor_msgs.msg import CameraInfo, Image
from cv_bridge import CvBridge
import cv2
from tf import transformations as tr
import numpy as np

class ApriltagsTesterNode(unittest.TestCase):
    def setup(self):
        # Setup the node
        rospy.init_node('apriltags_tester_node', anonymous=False)
        self.msg_received = False
        self.msg_tags = AprilTagDetectionArray()

        # Setup the publisher and subscriber
        self.pub_raw  = rospy.Publisher("~image_rect", Image, queue_size=1, latch=True)
        self.pub_info  = rospy.Publisher("~camera_info", CameraInfo, queue_size=1, latch=True)
        self.sub_tag = rospy.Subscriber( "~apriltags", AprilTagDetectionArray, self.tagCallback)

        # Wait for the node  to finish starting up
        timeout = rospy.Time.now() + rospy.Duration(5) # Wait at most 5 seconds for the node to come up
        while

### apriltags_tester_node.test

In [ ]:
!cat /content/catkin_ws/src/apriltags_ros_test/tests/apriltags_tester_node.test

<launch>
    <arg name="pkg_name" value="apriltags_ros"/>
    <arg name="node_name" default="apriltags_tester_node"/>

    <arg name="veh" default="testbot" doc="Name of vehicle. ex: megaman"/>
    <!--<arg name="filename" default="$(find apriltags_ros_test)/tests/20160408-226-intersection_apriltags_24_60-amadobot.png" />-->
    <arg name="filename" default="$(find apriltags_ros_test)/tests/20160508-apriltag108-starducks.png" />
    <!--<arg name="filename" default="$(find apriltags_ros_test)/tests/20210315-apriltag163-stop.jpg" />-->

    <!-- Run the node -->
    <include file="$(find apriltags_ros_test)/launch/apriltag_detector_node.launch">
        <arg name="veh" value="$(arg veh)"/>
    </include>

    <!-- Run unit test -->
    <remap from="apriltags_tester_node/image_rect" to="image_rect" />
    <remap from="apriltags_tester_node/camera_info" to="camera_info" />
    <remap from="apriltags_tester_node/apriltags" to="tag_detections" />
    <test test-name="apriltags_tester_node" 

### Run rostest

Run rostest for apriltags_ros_test apriltags_tester_node.test

In [ ]:
!source /content/catkin_ws/devel/setup.bash && rostest apriltags_ros_test apriltags_tester_node.test

... logging to /root/.ros/log/rostest-166ed9e33119-9016.log
[ROSUNIT] Outputting test results to /root/.ros/test_results/apriltags_ros_test/rostest-tests_apriltags_tester_node.xml
[ INFO] [1645880466.360412285]: Loaded tag config: 1, size: 0.065, frame_name: viewed_tag_1
[ INFO] [1645880466.363685635]: Loaded tag config: 2, size: 0.065, frame_name: viewed_tag_2
[ INFO] [1645880466.363935349]: Loaded tag config: 3, size: 0.065, frame_name: viewed_tag_3
[ INFO] [1645880466.364006595]: Loaded tag config: 4, size: 0.065, frame_name: viewed_tag_4
[ INFO] [1645880466.364071643]: Loaded tag config: 5, size: 0.065, frame_name: viewed_tag_5
[ INFO] [1645880466.364133534]: Loaded tag config: 6, size: 0.065, frame_name: viewed_tag_6
[ INFO] [1645880466.364187739]: Loaded tag config: 7, size: 0.065, frame_name: viewed_tag_7
[ INFO] [1645880466.364240571]: Loaded tag config: 8, size: 0.065, frame_name: viewed_tag_8
[ INFO] [1645880466.364294389]: Loaded tag config: 9, size: 0.065, frame_name: viewe